## 0. Imports and Constants
- Select user before proceeding

In [74]:
############## AUTORELOAD MAGIC ###################
%load_ext autoreload
%autoreload 2
###################################################

############## FUNDAMENTAL MODULES ################
import json
from PIL import Image
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pickle
import re
 ##################################################

############## TASK-SPECIFIC MODULES ##############
sys.path.append(os.path.join(os.getcwd(),"src"))
from vanilla_dataset import VanillaDataset
from vanilla_nn import VanillaNN
from trainer import Trainer
###################################################


####################### CONSTANTS ########################
SPLITS = ["train", "dev", "test", "gold"]
TRAIN, DEV, TEST, TXT, IMG = "train", "dev", "test", "txt", "img"
FE_METHODS = ["txt_embeddings", "img_embeddings", "concat", "sum", "mean", "hadamard"]
#FE_METHODS += ["concat_cos", "sum_cos", "mean_cos", "hadamard_cos"]
##########################################################

############## DATA SCIENCE & ML MODULES #################
from transformers import CLIPTokenizerFast, CLIPProcessor, CLIPModel
import torch
import pandas as pd
from scipy import stats
from sklearn.svm import SVC
from sklearn.metrics import classification_report
##########################################################

####################### SELECT ###########################
users = ["patriziopalmisano", "onurdenizguler", "jockl"]
user = users[2] # SELECT USER
version = "v2" # SELECT DATASET VERSION
dataset_version = version
##########################################################

if user in users[:2]:
    data_dir = f"/Users/{user}/Library/CloudStorage/GoogleDrive-check.worthiness@gmail.com/My Drive/data/CT23_1A_checkworthy_multimodal_english_{version}"
    cw_dir = f"/Users/{user}/Library/CloudStorage/GoogleDrive-check.worthiness@gmail.com/My Drive"

else:
    data_dir = f"/home/jockl/Insync/check.worthiness@gmail.com/Google Drive/data/CT23_1A_checkworthy_multimodal_english_{dataset_version}"
    cw_dir = "/home/jockl/Insync/check.worthiness@gmail.com/Google Drive"

features_dir = f"{data_dir}/features"
labels_dir = f"{data_dir}/labels"
models_dir = f"{cw_dir}/models/vanillann_hyperparam_search"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Load the Datasets

In [75]:
split_to_labels = {split: 
                   np.load(f"{labels_dir}/{split}_labels_{dataset_version}.pickle", allow_pickle=True) if split != "gold" else np.load(f"{labels_dir}/{split}_labels.pickle", allow_pickle=True)
                   for split in SPLITS}

method_to_split_to_data = {fe_method: {
                                    split: 
                                    np.load(f"{features_dir}/{fe_method}/{fe_method}_{split}_{dataset_version}.pickle", allow_pickle=True) if split != "gold" else np.load(f"{features_dir}/{fe_method}/{fe_method}_{split}.pickle", allow_pickle=True)
                                    for split in SPLITS
                                    } 
                            for fe_method in FE_METHODS}

method_to_split_to_dataset = {fe_method: {
                                        split:
                                        VanillaDataset(method_to_split_to_data[fe_method][split], split_to_labels[split]) 
                                        for split in SPLITS
                                        } 
                                for fe_method in FE_METHODS}

# 2. Hyperparam Search

In this section, the same hyperparameter search is performed on different feature engineering methods. Every search features the following parameters:

In [76]:
# Hyperparams to search from
model_init_param_configs = [
    [128, 64, 32, 16],
    [128, 64, 32],
    [64, 32, 128],
    [128, 64],
    [64, 32],
    [32, 16],
    [64],
    [32],
    [16]
]
learning_rates = [1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
batch_sizes = [8, 16, 32, 64, 128]
num_epochs = 1000

In [77]:
# Select device
device = "cpu"

## 2.1 Hyperparam Search on Text Embeddings

In [78]:
# Select method 
dataset_method = "txt_embeddings"
dataset_dict = method_to_split_to_dataset[dataset_method]

# Select model architecture and model class
model = VanillaNN

# Instantiate a trainer on the selected model class, model architecture, and dataset
trainer_text = Trainer(model, dataset_method, dataset_dict)

In [79]:
# Perform hyper param search on the selected method
for model_config in model_init_param_configs:
    trainer_text.hyperparameter_search(model_init_params = model_config,
                                    device = device,
                                    learning_rates = learning_rates,
                                    batch_sizes = batch_sizes,
                                    num_epochs = num_epochs)

  0%|          | 1/1000 [00:00<02:49,  5.89it/s]

Early quitting at epoch: 16


  0%|          | 1/1000 [00:00<01:52,  8.91it/s]

Early quitting at epoch: 10


  0%|          | 0/1000 [00:00<?, ?it/s]

...

Early quitting at epoch: 10


  0%|          | 2/1000 [00:00<00:59, 16.83it/s]

Early quitting at epoch: 11


  1%|          | 10/1000 [00:00<01:02, 15.72it/s]

Early quitting at epoch: 10


In [80]:
# Save all trained models
dir = f"{models_dir}/{dataset_method}"
trainer_text.save_trained_models(dir=dir, to_be_saved_models=None)

## 2.2 Hyperparam Search on Concat Embeddings

In [81]:
# Select method 
dataset_method = "concat"
dataset_dict = method_to_split_to_dataset[dataset_method]

# Select model architecture and model class
model = VanillaNN

# Instantiate a trainer on the selected model class, model architecture, and dataset
trainer_concat = Trainer(model, dataset_method, dataset_dict)

In [82]:
# Perform hyper param search on the selected method
for model_config in model_init_param_configs:
    trainer_concat.hyperparameter_search(model_init_params = model_config,
                                    device = device,
                                    learning_rates = learning_rates,
                                    batch_sizes = batch_sizes,
                                    num_epochs = num_epochs)

  0%|          | 0/1000 [00:00<?, ?it/s]

Early quitting at epoch: 759


  0%|          | 0/1000 [00:00<?, ?it/s]

Early quitting at epoch: 12


  0%|          | 1/1000 [00:00<01:57,  8.54it/s]

...

Early quitting at epoch: 10


  0%|          | 2/1000 [00:00<01:04, 15.42it/s]

Early quitting at epoch: 10


  1%|          | 10/1000 [00:00<01:13, 13.55it/s]

Early quitting at epoch: 10


In [83]:
# Save all trained models
dir = f"{models_dir}/{dataset_method}"
trainer_concat.save_trained_models(dir=dir, to_be_saved_models=None)

## 2.3 Hyperparam Search on Mean Embeddings

In [84]:
# Select method 
dataset_method = "mean"
dataset_dict = method_to_split_to_dataset[dataset_method]

# Select model architecture and model class
model = VanillaNN

# Instantiate a trainer on the selected model class, model architecture, and dataset
trainer_mean = Trainer(model, dataset_method, dataset_dict)

In [85]:
# Perform hyper param search on the selected method
for model_config in model_init_param_configs:
    trainer_mean.hyperparameter_search(model_init_params = model_config,
                                    device = device,
                                    learning_rates = learning_rates,
                                    batch_sizes = batch_sizes,
                                    num_epochs = num_epochs)

  0%|          | 0/1000 [00:00<?, ?it/s]

Early quitting at epoch: 26


  0%|          | 0/1000 [00:00<?, ?it/s]

Early quitting at epoch: 13


  0%|          | 1/1000 [00:00<02:17,  7.29it/s]

...

Early quitting at epoch: 10


  0%|          | 2/1000 [00:00<00:53, 18.79it/s]

Early quitting at epoch: 11


  1%|          | 11/1000 [00:00<01:02, 15.87it/s]

Early quitting at epoch: 11


In [86]:
# Save all trained models
dir = f"{models_dir}/{dataset_method}"
trainer_mean.save_trained_models(dir=dir, to_be_saved_models=None)